In [1]:
# Imports + all input data + tunable constants.

from itertools import product as cartesian_product
import math
import pandas as pd
from IPython.display import display  # to show tables nicely in notebooks

# ---- Tunable constants (assumptions) ----
STAGING_DAYS_CD = 3   # average CD staging time in days (Little's Law for cross-dock)
GAMMA_RF = 5          # RF explosion factor: 1 reserve load -> 5 smaller forward loads

# ---- Sets ----
products = [1, 2, 3, 4, 5, 6]
flows = [1, 2, 3, 4]  # 1=CD, 2=R, 3=RF, 4=F
flow_name = {1: "CD", 2: "R", 3: "RF", 4: "F"}
areas = ["CD", "R", "F"]

# ---- Table 11.1 ----
# Treat "Annual demand" as unit loads per year (consistent with the cost tables)
D = {1: 10000, 2: 15000, 3: 25000, 4: 2000, 5: 1500, 6: 95000}   # annual demand (unit loads)
S = {1: 50, 2: 50, 3: 50, 4: 50, 5: 50, 6: 150}                 # order cost ($/order)
P = {1: 500, 2: 650, 3: 350, 4: 250, 5: 225, 6: 150}            # price per unit load ($)
c_rate = {i: 0.10 for i in products}                            # carrying cost rate
space = {1: 10, 2: 15, 3: 25, 4: 10, 5: 12, 6: 13}              # m² per unit load

# RF reserve dwell fraction (only relevant if assigned to RF)
# NOTE: As stated, only 3 (20%) and 6 (100%) have nonzero reserve dwell when in RF.
alpha_reserve_RF = {1: 0.0, 2: 0.0, 3: 0.20, 4: 0.0, 5: 0.0, 6: 1.0}

# ---- Table 11.2 (handling cost per unit load) ----
handling_table = {
    1: {1: 0.0707, 2: 0.0849, 3: 0.1061, 4: 0.0778},
    2: {1: 0.0203, 2: 0.2023, 3: 0.2023, 4: 0.2023},
    3: {1: 0.0267, 2: 0.0420, 3: 0.0054, 4: 0.0481},
    4: {1: 0.3354, 2: 0.5590, 3: 1.0062, 4: 0.0671},
    5: {1: 0.4083, 2: 0.6804, 3: 1.2248, 4: 0.8165},
    6: {1: 0.0726, 2: 0.0871, 3: 0.1088, 4: 0.0798},
}
c_handling = {(i, f): handling_table[i][f] for i in products for f in flows}

# ---- Table 11.3 (storage cost per unit load per year) ----
storage_table = {
    1: {1: 20, 2: 5,  3: 10, 4: 15},
    2: {1: 15, 2: 5,  3: 10, 4: 10},
    3: {1: 4,  2: 20, 3: 1,  4: 9},
    4: {1: 5,  2: 4,  3: 5,  4: 1},
    5: {1: 15, 2: 25, 3: 45, 4: 30},
    6: {1: 20, 2: 5,  3: 10, 4: 15},
}
c_storage = {(i, f): storage_table[i][f] for i in products for f in flows}

# ---- Table 11.4 + total capacity ----
area_bounds = {"CD": (0, 15000), "R": (35000, 75000), "F": (35000, 75000)}
TOTAL_WAREHOUSE_CAP = 100_000  # "up to"

# ---- Table 11.5 (levels) ----
levels = {"CD": 1, "R": 1, "F": 1}


In [2]:
# EOQ and derived average on-hand/dwell (per unit load)
def eoq(i: int) -> float:
    # H ($/load-year) = price * carrying rate
    H = P[i] * c_rate[i]
    return math.sqrt(2 * D[i] * S[i] / H)

def avg_on_hand(i: int) -> float:
    # Standard EOQ assumption: average cycle stock = Q/2
    return 0.5 * eoq(i)

def dwell_years(i: int) -> float:
    return avg_on_hand(i) / D[i]

def dwell_days(i: int) -> float:
    return dwell_years(i) * 365.0

df_eoq = pd.DataFrame(
    [{"Product": i,
      "EOQ (loads)": eoq(i),
      "Avg on-hand (loads)": avg_on_hand(i),
      "Avg dwell (days)": dwell_days(i)} for i in products]
)
display(df_eoq.round(3))


,Product,EOQ (loads),Avg on-hand (loads),Avg dwell (days)
0,1,141.421,70.711,2.581
1,2,151.911,75.955,1.848
2,3,267.261,133.631,1.951
3,4,89.443,44.721,8.162
4,5,81.650,40.825,9.934
5,6,1378.405,689.202,2.648


In [3]:
def avg_load_equivalents(i: int, f: int) -> float:
    """
    Average 'load equivalents' for product i in flow f,
    used for both storage cost scaling and m² consumption.
    """
    if f == 1:  # Cross-dock: Little's Law with staging days
        return D[i] * (dwell_days(i) / 365.0)
    elif f in (2, 4):  # Pure Reserve or pure Forward
        return eoq(i) / 2
    elif f == 3:  # RF: split between Reserve and Forward with explosion in forward
        alpha = alpha_reserve_RF[i]
        base = eoq(i)/2
        # total load-equivalents stored across areas
        return alpha * base + GAMMA_RF * (1 - alpha) * base
    else:
        raise ValueError("Unknown flow")

def area_use_by_area(i: int, f: int) -> dict:
    """
    m² used by product i in each functional area for flow f.
    """
    s = space[i] / levels["R"]  # levels are 1 here, kept for completeness
    use = {"CD": 0.0, "R": 0.0, "F": 0.0}
    if f == 1:
        use["CD"] = avg_load_equivalents(i, f) * s
    elif f == 2:
        use["R"]  = avg_load_equivalents(i, f) * s
    elif f == 4:
        use["F"]  = avg_load_equivalents(i, f) * s
    elif f == 3:
        base  = avg_on_hand(i)
        alpha = alpha_reserve_RF[i]
        I_R = alpha * base
        I_F_equiv = GAMMA_RF * (1 - alpha) * base
        use["R"] = I_R * s
        use["F"] = I_F_equiv * s
    return use

def area_coeff(i: int, f: int, a: str) -> float:
    """Coefficient for area capacity constraint: m² used in area a if product i chooses flow f."""
    return area_use_by_area(i, f)[a]

def annual_cost_for_product(i: int, f: int) -> dict:
    # Handling cost scales with annual throughput; table gives $ per unit load in flow f
    handling = D[i] * c_handling[(i, f)]
    # Storage cost scales with average load equivalents; table gives $ per load-year in flow f
    storage  = avg_load_equivalents(i, f) * c_storage[(i, f)]
    return {"handling": handling, "storage": storage, "total": handling + storage}

def compute_area_consumption(assignment: dict) -> dict:
    cons = {"CD": 0.0, "R": 0.0, "F": 0.0}
    for i, f in assignment.items():
        use = area_use_by_area(i, f)
        for a in areas:
            cons[a] += use[a]
    return cons

def choose_area_sizes(cons: dict):
    """
    Given consumption by area, pick the smallest feasible area sizes satisfying:
      - per-area lower/upper bounds (Table 11.4)
      - total cap <= TOTAL_WAREHOUSE_CAP
    """
    sizes = {}
    for a in areas:
        lb, ub = area_bounds[a]
        need = cons[a]
        size_a = max(lb, need)
        if size_a > ub + 1e-9:
            return False, None  # infeasible requested consumption vs bounds
        sizes[a] = size_a

    # Check overall cap
    if sum(sizes.values()) > TOTAL_WAREHOUSE_CAP + 1e-9:
        return False, None

    return True, sizes

def evaluate_assignment(assignment: dict):
    cons = compute_area_consumption(assignment)
    feas, sizes = choose_area_sizes(cons)
    rows = []
    total_cost = 0.0
    for i, f in assignment.items():
        comp = annual_cost_for_product(i, f)
        rows.append({
            "Product": i,
            "Flow": flow_name[f],
            "Handling Cost": comp["handling"],
            "Storage Cost":  comp["storage"],
            "Total Cost":    comp["total"],
        })
        total_cost += comp["total"]
    df = pd.DataFrame(rows)
    return feas, sizes, total_cost, df, cons


In [4]:
def compute_area_consumption(assignment:dict) -> dict:
    cons = {"CD":0.0, "R":0.0, "F":0.0}
    for i, f in assignment.items():
        use = area_use_by_area(i,f)
        for a in areas:
            cons[a] += use[a]
    return cons

def choose_area_sizes(cons:dict):
    sizes = {}
    total = 0.0
    for a in areas:
        lb, ub = area_bounds[a]
        need = cons[a]
        size_a = max(lb, need)
        if size_a > ub + 1e-9:
            return None
        sizes[a] = size_a
        total += size_a
    if total > TOTAL_WAREHOUSE_CAP + 1e-9:
        return None
    return sizes

def evaluate_assignment(assignment:dict):
    cons = compute_area_consumption(assignment)
    sizes = choose_area_sizes(cons)
    if sizes is None:
        return False, None, float("inf"), None, None

    rows = []
    total_cost = 0.0
    for i, f in assignment.items():
        comp = annual_cost_for_product(i,f)
        rows.append({
            "Product": i,
            "Flow": flow_name[f],
            "Handling Cost": comp["handling"],
            "Storage Cost":  comp["storage"],
            "Total Cost":    comp["total"]
        })
        total_cost += comp["total"]

    df = pd.DataFrame(rows)
    return True, sizes, total_cost, df, cons


In [5]:
try:
    import pulp as pl

    m = pl.LpProblem("ForwardReserve_AreaSizing", pl.LpMinimize)

    # Variables
    x = pl.LpVariable.dicts("x", (products, flows), lowBound=0, upBound=1, cat=pl.LpBinary)
    A = pl.LpVariable.dicts("A", areas, lowBound=0, cat=pl.LpContinuous)

    # Objective: sum_i sum_f x_if * (handling_i,f + storage_i,f)
    m += pl.lpSum(
        x[i][f] * (
            D[i] * c_handling[(i, f)] +
            avg_load_equivalents(i, f) * c_storage[(i, f)]
        )
        for i in products for f in flows
    )

    # Each product assigned to exactly one flow
    for i in products:
        m += pl.lpSum(x[i][f] for f in flows) == 1, f"assign_{i}"

    # Area capacity constraints: for each area a,
    #   sum_i sum_f x_if * area_coeff(i,f,a) <= A_a
    for a in areas:
        m += pl.lpSum(x[i][f] * area_coeff(i, f, a)
                       for i in products for f in flows) <= A[a], f"area_cap_{a}"

    # Area lower/upper bounds
    for a in areas:
        lb, ub = area_bounds[a]
        m += A[a] >= lb, f"lb_{a}"
        m += A[a] <= ub, f"ub_{a}"

    # Total building cap
    m += pl.lpSum(A[a] for a in areas) <= TOTAL_WAREHOUSE_CAP, "total_cap"

    # Solve
    _ = m.solve(pl.PULP_CBC_CMD(msg=False))

    # Report
    status = pl.LpStatus[m.status]
    print("Solver status:", status)
    if status != "Optimal":
        # Fall back: show best found (if any)
        pass

    # Extract solution
    sol_assign = {i: max(flows, key=lambda f: pl.value(x[i][f])) for i in products}
    sol_sizes  = {a: pl.value(A[a]) for a in areas}

    feasible, sized, tot_cost, df_cost, cons = evaluate_assignment(sol_assign)

    print("\n=== Optimal product-to-flow assignment ===")
    display(pd.DataFrame(
        [{"Product": i, "Assigned Flow": flow_name[sol_assign[i]]} for i in products]
    ).sort_values("Product"))

    print("\n=== Area consumption (m²) from assignment ===")
    display(pd.DataFrame([cons]).T.rename(columns={0: "Consumption (m²)"}))

    print("\n=== Chosen area sizes (m²) ===")
    display(pd.DataFrame([sol_sizes]).T.rename(columns={0: "Size (m²)"}))

    print("\n=== Annual cost breakdown ===")
    display(df_cost.sort_values("Product").reset_index(drop=True))
    print(f"\nTotal annual cost: {tot_cost:,.2f} $")

except Exception as e:
    print("PuLP not available or MILP solve skipped. Details:", e)


PuLP not available or MILP solve skipped. Details: No module named 'pulp'


# 11.5 Warehouse Block Layout Design (6-Department Starter)
**Scope (Robustness target):** Handle at least these 6 departments  
1) Inbound Dock, 2) Receiving/Staging, 3) Pallet Reserve Storage (Bulk),  
4) Packing / Wrap / Banding, 5) Outbound Staging (Parcel + 2-Man combined),  
6) Shipping Dock.

**Goal:** For three layout patterns (I, L, U), place rectangular departments inside a bounding facility, respect fixed dock positions, avoid overlap, and **minimize total flow-weighted rectilinear (Manhattan) travel** between department centroids.

**Modeling approach (MILP):**
- We translate Muther SLP adjacency codes (E/A/I/O/U) into **quantitative flows** (4/3/2/1/0). This follows the common practice of mapping qualitative closeness to pairwise interaction weights, consistent with the literature (e.g., Heragu et al., 2005).
- To keep areas **exact** yet linear, we precompute a small set of **aspect-ratio options** for each department. A single binary choice selects one (width, height) combination with `width × height = target_area`.
- **Non-overlap** is enforced with the classic disjunctive Big-M formulation using four binaries per pair (left/right/above/below) and `L+R+B+T = 1`.
- **Distance** is rectified via absolute-value linearization: introduce `dx_ij, dy_ij` with `dx_ij ≥ x_i − x_j` and `dx_ij ≥ x_j − x_i` (similarly for `dy_ij`); objective is ∑ f_ij (dx_ij + dy_ij).

**Patterns (dock placement constraints):**
- **I-shaped:** Inbound dock fixed to the **left wall (center)**; Shipping dock fixed to the **right wall (center)** of a long rectangle.
- **L-shaped:** Inbound fixed to the **bottom wall (left wing)**; Shipping fixed to the **right wall (top wing)** of a moderately rectangular block (we emulate an L-flow by orthogonal dock placement).
- **U-shaped:** Both docks on the **same wall (left)** at **lower** (Inbound) and **upper** (Shipping) segments, encouraging a U-flow.

**Assessment settings:**
- Gurobi requested; we set `MIPGap = 0.05` as a target.
- Clear visualizations using Matplotlib.

**Reference:**  
Heragu, S. S., Du, L., Mantel, R. J., & Schuur, P. C. (2005). *Mathematical model for warehouse design and product allocation*. IJPR, 43(2), 327–38. https://doi.org/10.1080/00207540412331285841

In [6]:
# If you use a different solver, adapt here; Gurobi requested by spec.
try:
    !pip install gurobipy
    # --- IGNORE ---
    import gurobipy as gp
    from gurobipy import GRB
except Exception as e:
    raise RuntimeError(
        "This notebook is written for Gurobi (gurobipy). "
        "Please install & license Gurobi to run. Error: {}".format(e)
    )

import math
import itertools
import matplotlib.pyplot as plt
from dataclasses import dataclass


Defaulting to user installation because normal site-packages is not writeable


## Data: Areas & SLP mapping

**Areas (m²)** from Table 11.6 (restricted to the 6-department subset; Outbound Staging is aggregated):
- Inbound Dock: 2,640  
- Receiving/Staging: 5,280  
- Pallet Reserve Storage (Bulk): 46,340  
- Packing / Wrap / Banding: 3,520  
- Outbound Staging (Parcel + 2-Man): 3,520 + 5,280 = **8,800**  
- Shipping Dock: 3,520

**SLP mapping to flows:**  
We convert Muther SLP codes to numeric flows as:  
`E=4, A=3, I=2, O=1, U=0`.  
For the aggregated **Outbound Staging**, where two original rows existed (Parcel & 2-Man), we take the **max** code against any counterpart (equivalent to assuming aggregation preserves the strongest interaction).  
This yields a chain of strong interactions (a classic flow):  
Inbound ↔ Receiving (E), Receiving ↔ Bulk (I), Bulk ↔ Packing (E), Packing ↔ Staging (E), Staging ↔ Shipping (E).  
All other pairs default to lower or zero flow if their codes were U/O in the SLP.

We will use these flows directly as **unit-load intensities**; with unit transport cost per meter, the objective becomes total flow-weighted distance.


In [7]:
# ===== REPLACE YOUR "Cell 4 — Data Initialization (Code)" WITH THIS BLOCK =====
import itertools

# ------- Departments & areas (m^2) : full list from Table 11.6 -------
AREAS = {
    "Cross-Dock": 3520,
    "Empty Pallets & Dunnage": 880,
    "Inbound Dock": 2640,
    "Maintenance & Battery Charge": 1320,
    "Outbound Staging — 2-Man Delivery": 5280,
    "Outbound Staging — Parcel": 3520,
    "Oversize/Non-Standard Storage": 2640,
    "Packing / Wrap / Banding": 3520,
    "Pallet Reserve Storage (Bulk)": 46340,
    "QA & Technical Test": 1760,
    "Receiving/Staging": 5280,
    "Returns & WEEE": 2640,
    "Shipping Dock": 3520,
    "Spare Parts & Accessories Cage": 440,
}
DEPTS = list(AREAS.keys())

# ------- SLP code weights (Muther legend) -> NON-NEGATIVE mapping -------
# NOTE: We NEVER use negative weights in the objective.
RAW_WEIGHT = {"E": 4, "A": 3, "I": 2, "O": 1, "U": 0, "X": 0}

# ------- Encode the full SLP adjacency matrix (only non-U/X entries listed) -------
# We list all pairs (i,j,code) where code ∈ {E,A,I,O}. U is default 0, X we will capture separately.
slp_nonU = [
    # Inbound Dock row
    ("Inbound Dock", "Receiving/Staging", "E"),

    # Receiving/Staging row
    ("Receiving/Staging", "QA & Technical Test", "A"),
    ("Receiving/Staging", "Cross-Dock", "A"),
    ("Receiving/Staging", "Pallet Reserve Storage (Bulk)", "I"),
    ("Receiving/Staging", "Empty Pallets & Dunnage", "I"),

    # QA & Technical Test row
    ("QA & Technical Test", "Receiving/Staging", "A"),
    ("QA & Technical Test", "Returns & WEEE", "I"),

    # Cross-Dock row
    ("Cross-Dock", "Receiving/Staging", "A"),
    ("Cross-Dock", "Outbound Staging — Parcel", "A"),
    ("Cross-Dock", "Outbound Staging — 2-Man Delivery", "A"),
    ("Cross-Dock", "Shipping Dock", "A"),

    # Pallet Reserve Storage (Bulk) row
    ("Pallet Reserve Storage (Bulk)", "Receiving/Staging", "I"),
    ("Pallet Reserve Storage (Bulk)", "Packing / Wrap / Banding", "E"),
    ("Pallet Reserve Storage (Bulk)", "Maintenance & Battery Charge", "O"),

    # Oversize/Non-Standard Storage row
    ("Oversize/Non-Standard Storage", "Packing / Wrap / Banding", "I"),

    # Packing / Wrap / Banding row
    ("Packing / Wrap / Banding", "Pallet Reserve Storage (Bulk)", "E"),
    ("Packing / Wrap / Banding", "Oversize/Non-Standard Storage", "I"),
    ("Packing / Wrap / Banding", "Outbound Staging — Parcel", "E"),
    ("Packing / Wrap / Banding", "Outbound Staging — 2-Man Delivery", "E"),
    ("Packing / Wrap / Banding", "Empty Pallets & Dunnage", "O"),
    ("Packing / Wrap / Banding", "Returns & WEEE", "O"),

    # Outbound Staging — Parcel row
    ("Outbound Staging — Parcel", "Cross-Dock", "A"),
    ("Outbound Staging — Parcel", "Packing / Wrap / Banding", "E"),
    ("Outbound Staging — Parcel", "Shipping Dock", "E"),

    # Outbound Staging — 2-Man Delivery row
    ("Outbound Staging — 2-Man Delivery", "Cross-Dock", "A"),
    ("Outbound Staging — 2-Man Delivery", "Packing / Wrap / Banding", "E"),
    ("Outbound Staging — 2-Man Delivery", "Shipping Dock", "E"),

    # Shipping Dock row
    ("Shipping Dock", "Cross-Dock", "A"),
    ("Shipping Dock", "Outbound Staging — Parcel", "E"),
    ("Shipping Dock", "Outbound Staging — 2-Man Delivery", "E"),

    # Empty Pallets & Dunnage row
    ("Empty Pallets & Dunnage", "Receiving/Staging", "I"),
    ("Empty Pallets & Dunnage", "Packing / Wrap / Banding", "O"),

    # Maintenance & Battery Charge row
    ("Maintenance & Battery Charge", "Pallet Reserve Storage (Bulk)", "O"),

    # Returns & WEEE row
    ("Returns & WEEE", "QA & Technical Test", "I"),
    ("Returns & WEEE", "Packing / Wrap / Banding", "O"),

    # Spare Parts & Accessories Cage row
    # (all U by the table; no entries here)
]

# If you *do* have any "X" pairs in your data, list them here; we won't use negative weights.
# Example: undesirable_pairs_codes = [("Dept A", "Dept B"), ...]
undesirable_pairs_codes = []  # empty because the provided table had no X entries

# ------- Build symmetric numeric flow matrix f[(i,j)] for i<j -------
# We also add a small epsilon (background flow) so "U" pairs still have a mild cost.
EPSILON = 0.05   # tiny baseline attraction per pair (tune 0.02–0.10)
ALPHA   = 1.0    # scale for the SLP weights; keep 1.0 to match earlier runs

flows = {}
for i, j in itertools.combinations(DEPTS, 2):
    flows[(i, j)] = EPSILON  # baseline cost so no one is completely free to drift

# apply E/A/I/O weights
for a, b, code in slp_nonU:
    if a not in AREAS or b not in AREAS:
        raise ValueError(f"Unknown department name in SLP pair: ({a}, {b})")
    i, j = sorted([a, b], key=lambda s: DEPTS.index(s))
    flows[(i, j)] = EPSILON + ALPHA * RAW_WEIGHT[code]

# collect X pairs (if any) for min-separation constraints inside the model
undesirable_pairs = []
for a, b in undesirable_pairs_codes:
    i, j = sorted([a, b], key=lambda s: DEPTS.index(s))
    undesirable_pairs.append((i, j))
    # If you also want X-pairs to have a *low* weight (not zero), uncomment:
    # flows[(i, j)] = EPSILON  # keep only the baseline cost

# ------- Diagnostics -------
total_area = sum(AREAS.values())
num_pairs = len(list(itertools.combinations(DEPTS, 2)))
nonzero_pairs = sum(1 for k, v in flows.items() if v > EPSILON + 1e-9)

print(f"Departments: {len(DEPTS)} (expected 14)")
print(f"Total area: {total_area:,.0f} m² (envelope should be ≳ this)")
print(f"Pairs: {num_pairs}, nonzero flows (E/A/I/O): {nonzero_pairs}, baseline-only pairs: {num_pairs - nonzero_pairs}")
print(f"EPSILON (baseline): {EPSILON}, SLP scale ALPHA: {ALPHA}")

# Spot-check a few key pairs:
for key in [
    ("Inbound Dock", "Receiving/Staging"),
    ("Packing / Wrap / Banding", "Outbound Staging — Parcel"),
    ("Outbound Staging — 2-Man Delivery", "Shipping Dock"),
    ("Pallet Reserve Storage (Bulk)", "Packing / Wrap / Banding"),
    ("QA & Technical Test", "Returns & WEEE"),
    ("Spare Parts & Accessories Cage", "Receiving/Staging"),  # will now be EPSILON, not 0
]:
    i, j = sorted(key, key=lambda s: DEPTS.index(s))
    print(f"{key[0]} <-> {key[1]} : flow {flows[(i,j)]:.2f}")

Departments: 14 (expected 14)
Total area: 83,300 m² (envelope should be ≳ this)
Pairs: 91, nonzero flows (E/A/I/O): 18, baseline-only pairs: 73
EPSILON (baseline): 0.05, SLP scale ALPHA: 1.0
Inbound Dock <-> Receiving/Staging : flow 4.05
Packing / Wrap / Banding <-> Outbound Staging — Parcel : flow 4.05
Outbound Staging — 2-Man Delivery <-> Shipping Dock : flow 4.05
Pallet Reserve Storage (Bulk) <-> Packing / Wrap / Banding : flow 4.05
QA & Technical Test <-> Returns & WEEE : flow 2.05
Spare Parts & Accessories Cage <-> Receiving/Staging : flow 0.05


## Facility footprint and dock placements (I / L / U)

We approximate each pattern with a **single rectangular envelope** of equal total area for fairness, differing only by **dock placement**:

- Total required area (departments only): 70,100 m².  
  We add ~20% space for aisles/circulation = **84,120 m²**.

We choose three envelopes near this area:
- **I-shaped (long):** `W=420 m, H=200 m` → 84,000 m² (AR ≈ 2.1)  
- **L-shaped (emulated via orthogonal dock placement):** `W=360 m, H=234 m` → 84,240 m² (AR ≈ 1.54)  
- **U-shaped (more square):** `W=290 m, H=290 m` → 84,100 m² (AR ≈ 1.0)

**Dock placement rules** (implemented by fixing (x,y) relationships):
- Left wall: `x = 0.5 * width`
- Right wall: `x = W - 0.5 * width`
- Bottom wall: `y = 0.5 * height`
- Top wall: `y = H - 0.5 * height`
- Center on a wall: use the wall rule above and set the other coordinate to `H/2` or `W/2` as needed.

Specifics:
- **I-shaped:** Inbound @ **left-wall center**, Shipping @ **right-wall center**
- **L-shaped:** Inbound @ **bottom-wall** (left wing: `x = 0.25W`), Shipping @ **right-wall** (upper wing: `y = 0.75H`)
- **U-shaped:** Both on **left wall**; Inbound @ lower (`y = 0.25H`), Shipping @ upper (`y = 0.75H`)


In [8]:
# === Bootstrap Helpers: run this once before calling build_and_solve_two_phase ===
import math
from dataclasses import dataclass

# 1) LayoutSpec (if missing)
if "LayoutSpec" not in globals():
    @dataclass
    class LayoutSpec:
        name: str
        W: float
        H: float
        dock_rules: dict

# 2) make_aspect_options (if missing) – moderate, well-behaved aspect set
if "make_aspect_options" not in globals():
    def make_aspect_options(area, ratios=(0.67, 1.0, 1.5, 2.0)):
        """
        Return (w, h) options with w*h == area using aspect ratio r = h/w.
        Keep this set small-ish for faster solves; widen only if infeasible.
        """
        opts = []
        for r in ratios:
            h = math.sqrt(area * r)
            w = area / h
            opts.append((w, h))
        return opts

# 3) Sanity checks for data dictionaries from your earlier cells
missing = []
for name in ("AREAS", "DEPTS", "flows"):
    if name not in globals():
        missing.append(name)

if missing:
    raise RuntimeError(
        "Missing required data structure(s): {}. "
        "Re-run your Data Initialization cell that defines AREAS, DEPTS, and flows (full SLP)."
        .format(", ".join(missing))
)

def _infeasible_result(layout, mdl):
    # try to write an IIS (already have diagnose_infeasibility helper)
    try:
        mdl.computeIIS()
        fn = f"IIS_{layout.name}.ilp"
        mdl.write(fn)
        print(f"[IIS] Model infeasible — IIS written to {fn}")
    except gp.GurobiError as e:
        print(f"[IIS] Failed to compute IIS: {e}")

    # return a full-shaped payload so draw_layout doesn't crash
    return {
        "name": layout.name,
        "status": GRB.INFEASIBLE,
        "W": layout.W, "H": layout.H,
        "x": {i: None for i in DEPTS},
        "y": {i: None for i in DEPTS},
        "w": {i: None for i in DEPTS},
        "h": {i: None for i in DEPTS},
        "obj": None,
        "gap": None,
        "bound": None,
    }

In [9]:
from dataclasses import dataclass
import math
import itertools
import time
import gurobipy as gp
from gurobipy import GRB
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

@dataclass
class LayoutSpec:
    name: str
    W: float
    H: float
    dock_rules: dict  # maps department -> {"x_rule": ..., "y_rule": ...}

def make_aspect_options(area):
    """
    Adaptive aspect sets:
    - Very large areas: a few options including near-square and mild elongations
    - Medium: 3 options
    - Small: 3 options (symmetry around 1.0)
    """
    if area >= 25000:
        ratios = (0.75, 1.00, 1.25, 1.50)         # 4 options for the huge bulk area
    elif area >= 6000:
        ratios = (0.67, 1.00, 1.50)               # 3 options
    else:
        ratios = (0.67, 1.00, 1.50)               # 3 options
    opts = []
    for r in ratios:
        h = math.sqrt(area * r)
        w = area / h
        opts.append((w, h))
    return opts

def build_and_solve_two_phase(layout, miph_gap_target=0.01, total_seconds=900,
                              phase1_seconds=300, seed=42, verbose=True, threads=None):
    """
    Two-phase MIP solve (warm-started):
      Phase 1: feasible incumbent fast (incumbent-first)
      Phase 2: tighten bound to reach ≤ miph_gap_target within remaining time
    Returns a dict with geometry and objective value (or Starts/None if no incumbent).
    """
    gp.setParam("OutputFlag", 1 if verbose else 0)

    mdl = gp.Model(f"WarehouseLayout_{layout.name}")

    # ---------- Variables ----------
    x = {i: mdl.addVar(lb=0.0, ub=layout.W, name=f"x[{i}]") for i in DEPTS}
    y = {i: mdl.addVar(lb=0.0, ub=layout.H, name=f"y[{i}]") for i in DEPTS}

    aspects = {i: make_aspect_options(AREAS[i]) for i in DEPTS}
    z = {(i,k): mdl.addVar(vtype=GRB.BINARY, name=f"z[{i},{k}]")
         for i in DEPTS for k,_ in enumerate(aspects[i])}
    w = {i: mdl.addVar(lb=0.0, ub=layout.W, name=f"w[{i}]") for i in DEPTS}
    h = {i: mdl.addVar(lb=0.0, ub=layout.H, name=f"h[{i}]") for i in DEPTS}

    for i in DEPTS:
        mdl.addConstr(gp.quicksum(z[(i,k)] for k in range(len(aspects[i]))) == 1)
        mdl.addConstr(w[i] == gp.quicksum(aspects[i][k][0] * z[(i,k)] for k in range(len(aspects[i]))))
        mdl.addConstr(h[i] == gp.quicksum(aspects[i][k][1] * z[(i,k)] for k in range(len(aspects[i]))))
        mdl.addConstr(x[i] >= 0.5 * w[i]);  mdl.addConstr(x[i] <= layout.W - 0.5 * w[i])
        mdl.addConstr(y[i] >= 0.5 * h[i]);  mdl.addConstr(y[i] <= layout.H - 0.5 * h[i])

    # ---------- Tighter Big-M (pair-specific) ----------
    min_w = {i: min(opt[0] for opt in aspects[i]) for i in DEPTS}
    min_h = {i: min(opt[1] for opt in aspects[i]) for i in DEPTS}

    L= {}; R= {}; B= {}; T= {}
    for i, j in itertools.combinations(DEPTS, 2):
        L[(i,j)] = mdl.addVar(vtype=GRB.BINARY)
        R[(i,j)] = mdl.addVar(vtype=GRB.BINARY)
        B[(i,j)] = mdl.addVar(vtype=GRB.BINARY)
        T[(i,j)] = mdl.addVar(vtype=GRB.BINARY)
        mdl.addConstr(L[(i,j)] + R[(i,j)] + B[(i,j)] + T[(i,j)] == 1)
        
    # ---------- Non-overlap disjunction (must-have) ----------
    Mx = layout.W
    My = layout.H
    sep = 0.0   # set to 0.0 first; if you want a buffer later, try 0.05–0.10

    for i, j in itertools.combinations(DEPTS, 2):
        # Ensure exactly one relation (you already have this equality earlier)
        # mdl.addConstr(L[(i,j)] + R[(i,j)] + B[(i,j)] + T[(i,j)] == 1)

        # i left of j
        mdl.addConstr(
            x[i] + 0.5*w[i] + sep <= x[j] - 0.5*w[j] + Mx*(1 - L[(i,j)]),
            name=f"left[{i}|{j}]"
        )
        # i right of j
        mdl.addConstr(
            x[j] + 0.5*w[j] + sep <= x[i] - 0.5*w[i] + Mx*(1 - R[(i,j)]),
            name=f"right[{i}|{j}]"
        )
        # i below j
        mdl.addConstr(
            y[i] + 0.5*h[i] + sep <= y[j] - 0.5*h[j] + My*(1 - B[(i,j)]),
            name=f"below[{i}|{j}]"
        )
        # i above j
        mdl.addConstr(
            y[j] + 0.5*h[j] + sep <= y[i] - 0.5*h[i] + My*(1 - T[(i,j)]),
            name=f"above[{i}|{j}]"
        )



    # ---------- Distance linearization ----------
    dx = {}; dy = {}
    for i, j in itertools.combinations(DEPTS, 2):
        dx[(i,j)] = mdl.addVar(lb=0.0)
        dy[(i,j)] = mdl.addVar(lb=0.0)
        mdl.addConstr(dx[(i,j)] >= x[i] - x[j])
        mdl.addConstr(dx[(i,j)] >= x[j] - x[i])
        mdl.addConstr(dy[(i,j)] >= y[i] - y[j])
        mdl.addConstr(dy[(i,j)] >= y[j] - y[i])

    # ---------- Strengthening: link dx, dy to disjunction ----------
    Mx = layout.W
    My = layout.H
    for i, j in itertools.combinations(DEPTS, 2):
        # If i is left or right of j (horizontal separation), dx must cover half-widths
        mdl.addConstr(
            dx[(i,j)] >= 0.5*(w[i] + w[j]) - Mx*(B[(i,j)] + T[(i,j)]),
            name=f"dx_link[{i}|{j}]"
        )
        # If i is below or above j (vertical separation), dy must cover half-heights
        mdl.addConstr(
            dy[(i,j)] >= 0.5*(h[i] + h[j]) - My*(L[(i,j)] + R[(i,j)]),
            name=f"dy_link[{i}|{j}]"
        )

    # ---------- Dock rules ----------
    def apply_rule(axis, dept, rule):
        if axis == "x":
            if rule == "left":
                mdl.addConstr(x[dept] == 0.5 * w[dept])
            elif rule == "right":
                mdl.addConstr(x[dept] == layout.W - 0.5 * w[dept])
            elif rule == "center_x":
                mdl.addConstr(x[dept] == layout.W / 2.0)
            elif isinstance(rule, (int,float)):
                mdl.addConstr(x[dept] == float(rule))
        else:
            if rule == "bottom":
                mdl.addConstr(y[dept] == 0.5 * h[dept])
            elif rule == "top":
                mdl.addConstr(y[dept] == layout.H - 0.5 * h[dept])
            elif rule == "center_y":
                mdl.addConstr(y[dept] == layout.H / 2.0)
            elif isinstance(rule, (int,float)):
                mdl.addConstr(y[dept] == float(rule))

    for dept, rules in layout.dock_rules.items():
        if "x_rule" in rules: apply_rule("x", dept, rules["x_rule"])
        if "y_rule" in rules: apply_rule("y", dept, rules["y_rule"])
        
    def add_order_chain(mdl, x, W, enable=True, margin_frac=0.03):
        """Soft left-to-right precedence: x[u] <= x[v] + margin."""
        if not enable:
            return
        margin = margin_frac * W

        def _le(a, b):
            if (a in DEPTS) and (b in DEPTS):
                mdl.addConstr(x[a] <= x[b] + margin, name=f"order_{a}_le_{b}")

        ORDER = [
            ("Inbound Dock", "Receiving/Staging"),
            ("Receiving/Staging", "Pallet Reserve Storage (Bulk)"),
            ("Pallet Reserve Storage (Bulk)", "Packing / Wrap / Banding"),
            ("Packing / Wrap / Banding", "Outbound Staging — Parcel"),
            ("Outbound Staging — Parcel", "Shipping Dock"),
        ]
        for u, v in ORDER:
            _le(u, v)
    
        # after dock rules (inside build_and_solve_two_phase)
    order_enable = (layout.name != "U-shaped")   # disable for U-shaped
    add_order_chain(mdl, x, layout.W, enable=order_enable, margin_frac=0.02)  # 2% for I/L; off for U
    


    # ---------- Objective ----------
    obj = gp.quicksum(flows[(i,j)] * (dx[(i,j)] + dy[(i,j)]) for i, j in itertools.combinations(DEPTS, 2))
    mdl.setObjective(obj, GRB.MINIMIZE)

    # ---------- Branch priorities ----------
    for d in DEPTS:
        x[d].BranchPriority = 10; y[d].BranchPriority = 10
    for (i,k), zvar in z.items():
        zvar.BranchPriority = 20 if AREAS[i] > 10000 else 5

    # ---------- Phase 1: incumbent-first ----------
    t0 = time.time()
    if threads is not None:
        mdl.setParam("Threads", int(threads))
        
    mdl.setParam("Seed", seed)
    mdl.setParam("TimeLimit", max(5, int(phase1_seconds)))
    mdl.setParam("MIPGap", 0.05)
    mdl.setParam("MIPFocus", 1)
    mdl.setParam("Heuristics", 0.35)
    mdl.setParam("Cuts", 1)
    mdl.setParam("Presolve", 2)
    mdl.optimize()
    
    if mdl.status == GRB.INFEASIBLE:
        return _infeasible_result(layout, mdl)

    # Warm-start data
    has_inc = (getattr(mdl, "SolCount", 0) or 0) > 0
    inc_vals = {}
    if has_inc:
        for v in mdl.getVars():
            inc_vals[v.VarName] = v.X

    # ---------- Phase 2: bound-oriented ----------
    target_hit = (has_inc and hasattr(mdl, "MIPGap") and mdl.MIPGap is not None and mdl.MIPGap <= miph_gap_target + 1e-9)
    if not target_hit:
        elapsed = time.time() - t0
        remaining = max(10, int(total_seconds - elapsed))
    if threads is not None:
        mdl.setParam("Threads", int(threads))

        mdl.reset()  # clears tree; keep model; warm start below
        mdl.setParam("TimeLimit", remaining)
        mdl.setParam("MIPGap", miph_gap_target)
        mdl.setParam("MIPFocus", 3)
        mdl.setParam("Heuristics", 0.05)
        mdl.setParam("Cuts", 2)
        mdl.setParam("Presolve", 2)
        mdl.setParam("Method", 2)      # barrier at root
        mdl.setParam("Crossover", 0)
        mdl.setParam("ZeroObjNodes", 1)
        mdl.setParam("IntegralityFocus", 1)
        # mdl.setParam("NodefileStart", 0.5)  # optional memory safeguard
        
        if mdl.status == GRB.INFEASIBLE:
            return _infeasible_result(layout, mdl)

        # MIP start
        if inc_vals:
            for v in mdl.getVars():
                if v.VarName in inc_vals:
                    v.Start = inc_vals[v.VarName]

        mdl.optimize()

    # ---------- Collect solution (robust) ----------
    has_sol = (getattr(mdl, "SolCount", 0) or 0) > 0

    def _val(v):
        if has_sol:
            try:
                return v.X
            except Exception:
                pass
        return getattr(v, "Start", None)

    best_obj = mdl.objVal if has_sol else None
    best_bound = getattr(mdl, "ObjBound", None)
    gap_val = None
    try:
        if has_sol and best_obj not in (None, 0) and best_bound is not None:
            gap_val = abs((best_bound - best_obj) / (best_obj + 1e-10))
        elif (best_obj is not None) and (best_bound is not None) and best_obj != 0:
            gap_val = abs((best_bound - best_obj) / (abs(best_obj) + 1e-10))
    except Exception:
        pass

    sol = {
        "obj": best_obj,
        "W": layout.W, "H": layout.H,
        "x": {i: _val(x[i]) for i in DEPTS},
        "y": {i: _val(y[i]) for i in DEPTS},
        "w": {i: _val(w[i]) for i in DEPTS},
        "h": {i: _val(h[i]) for i in DEPTS},
        "status": mdl.status,
        "gap": (mdl.MIPGap if has_sol and hasattr(mdl, "MIPGap") else gap_val),
        "bound": best_bound,
        "name": layout.name
    }

    # Quick log
    status_map = {
        GRB.OPTIMAL: "OPTIMAL",
        GRB.TIME_LIMIT: "TIME_LIMIT",
        GRB.SUBOPTIMAL: "SUBOPTIMAL",
        GRB.INFEASIBLE: "INFEASIBLE",
        GRB.INF_OR_UNBD: "INF_OR_UNBD",
        GRB.INTERRUPTED: "INTERRUPTED",
        GRB.USER_OBJ_LIMIT: "USER_OBJ_LIMIT",
    }
    print(f"[{layout.name}] status={status_map.get(mdl.status, mdl.status)}, "
          f"incumbent={best_obj}, bound={best_bound}, gap={sol['gap']}")
    return sol

def _solve_one(spec, threads):
    """
    Small wrapper so it can be pickled and run in a separate process.
    """
    # Re-imports inside worker (multiprocessing best practice)
    import gurobipy as gp  # noqa: F401
    # Call your solver with requested threads
    res = build_and_solve_two_phase(
        spec,
        miph_gap_target=0.01,
        total_seconds=3600,
        phase1_seconds=1200,
        verbose=True,
        # pass through threads to solver
        # (ensure your function signature has `threads=None` and uses it)
        threads=threads,
    )
    return spec.name, res

# 2) Compute thread split
CPU_TOTAL = os.cpu_count() or 6
RESERVE_CORES = 0               # change if you want to keep cores free
usable = max(1, CPU_TOTAL - RESERVE_CORES)
threads_per_model = max(1, usable // 3)

print(f"Detected {CPU_TOTAL} logical cores → allocating {threads_per_model} threads per model.")

Detected 144 logical cores → allocating 48 threads per model.


## Visualization helper

We draw the facility boundary and the placed rectangles (department blocks) with labels and dimensions.


In [10]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import textwrap
from collections import OrderedDict

# ------------------ Global styling ------------------
mpl.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 260,
    "axes.titlesize": 16,
    "axes.labelsize": 11,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "font.size": 10,
    "axes.edgecolor": "#1c1c1c",
    "axes.linewidth": 1.4,
})

PALETTE = [
    "#4C78A8", "#F58518", "#54A24B", "#E45756", "#72B7B2",
    "#EECA3B", "#B279A2", "#FF9DA6", "#9D755D", "#BAB0AC",
    "#2790C3", "#E69F00", "#009E73", "#CC79A7"
]

def _dept_colors(depts):
    return {d: PALETTE[i % len(PALETTE)] for i, d in enumerate(depts)}

def _nice_dim(w, h):
    return f"{w:.1f}×{h:.1f} m"

def _halo_text(ax, x, y, s, **kw):
    # text with a white halo for readability
    text = ax.text(x, y, s, **kw)
    import matplotlib.patheffects as patheffects
    text.set_path_effects([
        patheffects.withStroke(linewidth=3, foreground="white")
    ])
    return text

def _make_numbered_legend(ax, id_map, color_map, ncols=2, title="Departments"):
    """
    Build a compact legend that works on older Matplotlib (uses ncol, not ncols).
    """
    handles, labels = [], []
    for d, idx in id_map.items():
        patch = mpl.patches.Patch(
            facecolor=color_map[d], edgecolor="#222", label=f"{idx}. {d}", alpha=0.45
        )
        handles.append(patch)
        labels.append(f"{idx}. {d}")

    # Use ncol (older Matplotlib); ignore unknown kwargs defensively
    legend_kwargs = dict(
        loc="center left",
        bbox_to_anchor=(1.02, 0.5),
        title=title,
        frameon=True,
        borderpad=0.6,
        labelspacing=0.45,
        ncol=ncols,               # <-- use ncol here
        columnspacing=0.8,
        handlelength=1.2,
        fontsize=9
    )
    try:
        leg = ax.legend(handles, labels, **legend_kwargs)
    except TypeError:
        # super old fallback (strip possibly unsupported args)
        legend_kwargs.pop("columnspacing", None)
        legend_kwargs.pop("handlelength", None)
        legend_kwargs.pop("fontsize", None)
        leg = ax.legend(handles, labels, **legend_kwargs)

    leg.get_frame().set_facecolor("white")
    leg.get_frame().set_edgecolor("#ddd")
    return leg

def draw_layout(solution,
                title=None,
                figsize=(12, 7.2),
                legend=True,
                legend_ncols=2,
                show_dims_top_k=4,
                fname=None):
    
    # Guard: infeasible or missing geometry
    if solution.get("status") == gp.GRB.INFEASIBLE or solution.get("W") is None:
        fig, ax = plt.subplots(figsize=figsize)
        ax.set_title(title or f"{solution.get('name', 'Layout')} — INFEASIBLE")
        ax.axis("off")
        ax.text(0.5, 0.5,
                f"{solution.get('name','Layout')} is INFEASIBLE\n"
                f"(see IIS file in working directory)",
                ha="center", va="center", fontsize=14)
        plt.tight_layout()
        if fname: plt.savefig(fname, dpi=200, bbox_inches="tight")
        plt.show()
        return
    """
    Clean, professional plot:
    - Numbered badges inside each block (no long text)
    - Legend maps number -> department
    - Dimensions shown only for top-k largest areas (plus any > min_area_to_tag)
    """
    W, H = solution["W"], solution["H"]
    x_s, y_s, w_s, h_s = solution["x"], solution["y"], solution["w"], solution["h"]

    # Establish deterministic department order and numeric IDs
    # (sorted by area descending → big first)
    areas = {d: w_s[d] * h_s[d] for d in DEPTS}
    order = sorted(DEPTS, key=lambda d: areas[d], reverse=True)
    id_map = OrderedDict((d, i+1) for i, d in enumerate(order))

    color_map = _dept_colors(DEPTS)

    fig, ax = plt.subplots(figsize=figsize)
    ax.set_xlim(0, W)
    ax.set_ylim(0, H)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlabel("X (m)")
    ax.set_ylabel("Y (m)")

    # Title with objective + gap on a second line in smaller font
    main_title = title or solution.get("name", "Layout")
    subbits = []
    if solution.get("obj") is not None:
        subbits.append(f"Objective (flow-weighted m): {solution['obj']:,.0f}")
    if solution.get("gap") is not None and solution["gap"] >= 0:
        subbits.append(f"MIP gap: {solution['gap']*100:.1f}%")
    subtitle = " • ".join(subbits)
    ax.set_title(main_title + ("\n" + subtitle if subtitle else ""), pad=12)

    # Facility boundary
    ax.add_patch(plt.Rectangle((0, 0), W, H, fill=False, lw=2.2, ec="#1b1b1b"))

    # Draw rectangles (big to small)
    for d in order:
        cx, cy, w, h = x_s[d], y_s[d], w_s[d], h_s[d]
        x0, y0 = cx - w/2.0, cy - h/2.0

        fc = color_map[d]
        ec = "#1b1b1b"

        # block
        rect = plt.Rectangle((x0, y0), w, h,
                             facecolor=fc, edgecolor=ec,
                             linewidth=1.6, alpha=0.16, zorder=2)
        ax.add_patch(rect)

        # numeric badge (centered, always horizontal)
        idx = id_map[d]
        # badge box behind the number for contrast
        badge = mpl.patches.FancyBboxPatch(
            (cx, cy), 1, 1,  # dummy, we place with transform below
            boxstyle="round,pad=0.25,rounding_size=0.8",
            fc="white", ec="#333", lw=0.9, zorder=5, transform=None, visible=False
        )
        # Instead of drawing a box, just draw a haloed number for minimal clutter
        _halo_text(ax, cx, cy, f"{idx}",
                   ha="center", va="center", fontsize=max(min(min(w,h)/7.0, 13), 9),
                   color="#111", weight="bold", zorder=6)

    # Dimension tags for largest K
    topK = set(order[:max(1, show_dims_top_k)])
    # Also tag any block occupying > 12% of facility area
    min_area_to_tag = 0.12 * (W * H)
    for d in order:
        cx, cy, w, h = x_s[d], y_s[d], w_s[d], h_s[d]
        x0, y0 = cx - w/2.0, cy - h/2.0
        if (d in topK) or (w*h >= min_area_to_tag):
            ax.text(x0 + 3, y0 + h - 3, _nice_dim(w, h),
                    ha="left", va="top", fontsize=9.0, color="#222",
                    bbox=dict(boxstyle="round,pad=0.22", fc="white", ec="none", alpha=0.75),
                    zorder=7)

    # Add neat grid
    ax.grid(True, which="both", alpha=0.14, linewidth=0.9)
    ax.tick_params(length=0)

    # Legend outside
    if legend:
        _make_numbered_legend(ax, id_map, color_map, ncols=legend_ncols, title="Departments")

    plt.tight_layout()
    if fname:
        plt.savefig(fname, bbox_inches="tight")
    plt.show()


## Define I, L, and U patterns (dock placements)

We plug the dock rules into `LayoutSpec`:
- **I:** Inbound → left wall + centered vertically; Shipping → right wall + centered vertically
- **L:** Inbound → bottom wall near left wing (`x = 0.25W`); Shipping → right wall near upper wing (`y = 0.75H`)
- **U:** Both docks on **left wall**; Inbound lower (`0.25H`), Shipping upper (`0.75H`)


In [11]:
# Recomputed totals: all 14 departments sum to 83,300 m²
# Use ≈ +30% slack for aisles/circulation → target ≈ 108–112k m²

I_layout = LayoutSpec(
    name="I-shaped",
    W=520.0, H=210.0,   # 109,200 m² (long)
    dock_rules={
        # Fix only to the wall; leave the orthogonal coordinate free.
        "Inbound Dock":  {"x_rule": "left"},   # y is free
        "Shipping Dock": {"x_rule": "right"},  # y is free
    },
)

L_layout = LayoutSpec(
    name="L-shaped",
    W=400.0, H=280.0,   # 112,000 m² (more rectangular)
    dock_rules={
        "Inbound Dock":  {"y_rule": "bottom"},     # x is free (bottom wall)
        "Shipping Dock": {"x_rule": "right"},      # y is free (right wall)
    },
)

U_layout = LayoutSpec(
    name="U-shaped",
    W=330.0, H=330.0,   # 108,900 m² (near-square)
    dock_rules={
        "Inbound Dock":  {"x_rule": "left"},   # both on left wall, y free
        "Shipping Dock": {"x_rule": "left"},
    },
)


## Solve the three models

We target `MIPGap ≤ 5%`. Set `verbose=True` to inspect Gurobi logs if desired.


In [12]:
#sol_I = build_and_solve_two_phase(I_layout, miph_gap_target=0.01, total_seconds=3600, phase1_seconds=1200, verbose=True)
#sol_L = build_and_solve_two_phase(L_layout, miph_gap_target=0.01, total_seconds=3600, phase1_seconds=1200, verbose=True)
#sol_U = build_and_solve_two_phase(U_layout, miph_gap_target=0.01, total_seconds=3600, phase1_seconds=1200, verbose=True)

# 3) Launch in parallel
futures = {}
with ProcessPoolExecutor(max_workers=3) as ex:
    futures[ex.submit(_solve_one, I_layout, threads_per_model)] = "I"
    futures[ex.submit(_solve_one, L_layout, threads_per_model)] = "L"
    futures[ex.submit(_solve_one, U_layout, threads_per_model)] = "U"

    results = {}
    for fut in as_completed(futures):
        tag = futures[fut]
        try:
            name, sol = fut.result()
            results[tag] = sol
            print(f"[{name}] done → status={sol.get('status')}, obj={sol.get('obj')}, gap={sol.get('gap')}")
        except Exception as e:
            print(f"[{tag}] failed:", e)

Restricted license - for non-production use only - expires 2026-11-23
Set parameter OutputFlag to value 1
Restricted license - for non-production use only - expires 2026-11-23
Set parameter OutputFlag to value 1
Set parameter Threads to value 48
Set parameter Seed to value 42
Restricted license - for non-production use only - expires 2026-11-23
Set parameter TimeLimit to value 1200
Set parameter OutputFlag to value 1
Set parameter MIPGap to value 0.05
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.35
Set parameter Cuts to value 1
Set parameter Presolve to value 2
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

Set parameter Threads to value 48
CPU model: Intel(R) Xeon(R) Platinum 8352V CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 72 physical cores, 144 logical processors, using up to 48 threads
Set parameter Seed to value 42

Set parameter TimeLimit to value 1200
Set parameter MIPGap to value 0.05
Non-defa

## Visualize the I / L / U layouts
Each plot includes the facility boundary, department rectangles, and dimensions. The overlay text shows the flow-weighted objective and (if provided) the MIP gap.


In [13]:
# 4) Optional: draw if feasible
if "I" in results:
    draw_layout(results["I"], title="I-Shaped Layout — All Departments", fname="layout_I_parallel.png")
if "L" in results:
    draw_layout(results["L"], title="L-Shaped Layout — All Departments", fname="layout_L_parallel.png")
if "U" in results:
    draw_layout(results["U"], title="U-Shaped Layout — All Departments", fname="layout_U_parallel.png")

## Quick comparison
We list objective values (lower is better).


In [14]:
summary = [
    (sol_I["name"], sol_I["obj"], sol_I.get("gap", None)),
    (sol_L["name"], sol_L["obj"], sol_L.get("gap", None)),
    (sol_U["name"], sol_U["obj"], sol_U.get("gap", None)),
]
for name, obj, gap in summary:
    if obj is None:
        print(f"{name:10s} : infeasible or no solution")
    else:
        gap_txt = f" (gap {gap*100:.1f}%)" if (gap is not None and gap >= 0) else ""
        print(f"{name:10s} : {obj:,.0f}{gap_txt}")


NameError: name 'sol_I' is not defined